<a href="https://colab.research.google.com/github/christophermalone/DSCI325/blob/main/NLP_Twitter_NoAccessKeyInfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing - Twitter

## Setting up Twitter Side

Go to Twitter Development website and setup an new app.

<p align='center'><img src="https://drive.google.com/uc?export=view&id=" width="25%" height="25%"></img></p>


## Setting up Python

In [ ]:
import os
import tweepy
import pandas as pd

# specify keys/secrets from Twitter App
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
results = api.search(q='rittenhouse', count=1000)

json_data = [r._json for r in results]

#Convert json data to data.frame
Twitterdf = pd.json_normalize(json_data)
Twitterdf.head()

,created_at,id,id_str,text,truncated,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,entities.hashtags,entities.symbols,entities.user_mentions,entities.urls,metadata.iso_language_code,metadata.result_type,user.id,user.id_str,user.name,user.screen_name,user.location,user.description,user.url,user.entities.description.urls,user.protected,user.followers_count,user.friends_count,user.listed_count,user.created_at,...,quoted_status.user.verified,quoted_status.user.statuses_count,quoted_status.user.lang,quoted_status.user.contributors_enabled,quoted_status.user.is_translator,quoted_status.user.is_translation_enabled,quoted_status.user.profile_background_color,quoted_status.user.profile_background_image_url,quoted_status.user.profile_background_image_url_https,quoted_status.user.profile_background_tile,quoted_status.user.profile_image_url,quoted_status.user.profile_image_url_https,quoted_status.user.profile_banner_url,quoted_status.user.profile_link_color,quoted_status.user.profile_sidebar_border_color,quoted_status.user.profile_sidebar_fill_color,quoted_status.user.profile_text_color,quoted_status.user.profile_use_background_image,quoted_status.user.has_extended_profile,quoted_status.user.default_profile,quoted_status.user.default_profile_image,quoted_status.user.following,quoted_status.user.follow_request_sent,quoted_status.user.notifications,quoted_status.user.translator_type,quoted_status.user.withheld_in_countries,quoted_status.geo,quoted_status.coordinates,quoted_status.place,quoted_status.contributors,quoted_status.is_quote_status,quoted_status.retweet_count,quoted_status.favorite_count,quoted_status.favorited,quoted_status.retweeted,quoted_status.lang,retweeted_status.entities.media,retweeted_status.extended_entities.media,entities.media,extended_entities.media
0,Thu Dec 02 16:59:50 +0000 2021,1466452055636201474,1466452055636201474,RT @JackPosobiec: BREAKING: Epstein Victim Tak...,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,NaN,None,None,None,None,NaN,None,False,184,0,False,False,en,[],[],"[{'screen_name': 'JackPosobiec', 'name': 'Jack...",[],en,recent,1443545649308393473,1443545649308393473,Greens and Flowers,TruthSniffercom,Georgia,Moringa harvest 2021 +50 first year,None,[],False,2,55,0,Thu Sep 30 11:58:21 +0000 2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Thu Dec 02 16:59:48 +0000 2021,1466452049172697090,1466452049172697090,Reactions to the Kyle Rittenhouse not guilty v...,False,"<a href=""http://publicize.wp.com/"" rel=""nofoll...",NaN,None,NaN,None,None,None,None,NaN,None,False,0,0,False,False,en,[],[],[],"[{'url': 'https://t.co/jBOGZHGO2e', 'expanded_...",en,recent,49404200,49404200,Charles Myrick -CEO,medicinehelp,"Atlanta,GA",American Consultants Rx is focused on helping ...,https://t.co/IEWbNbyI7E,[],False,1348,1848,54,Sun Jun 21 19:35:30 +0000 2009,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Thu Dec 02 16:59:46 +0000 2021,1466452041195134977,1466452041195134977,"RT @JonathanTurley: During the trial, NBC rele...",False,"<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,None,None,None,None,NaN,None,False,640,0,False,False,en,[],[],"[{'screen_name': 'JonathanTurley', 'name': 'Jo...",[],en,recent,221876632,221876632,Lori Guiver,Oh4aHyphensSake,,"I've never seen, heard, nor smelled an issue t...",None,[],False,253,712,9,Wed Dec 01 21:27:23 +0000 2010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Thu Dec 02 16:59:46 +0000 2021,14664520411365

In [ ]:
Twitterdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 230 entries, created_at to extended_entities.media
dtypes: bool(18), float64(46), int64(9), object(157)
memory usage: 167.5+ KB


In [ ]:
#Install dfply for dplyr like functionality in Python
!pip install dfply
from dfply import *

In [ ]:
#Piping in dfply and using filter_by() to grab requested rows.
Twitterdf_Reduced = ( Twitterdf
              >> select(X.created_at, X.id, X.text, X.retweet_count, X.favorite_count, X.possibly_sensitive)
              
            )
Twitterdf_Reduced.shape
Twitterdf_Reduced.head()

,created_at,id,text,retweet_count,favorite_count,possibly_sensitive
0,Thu Dec 02 16:59:50 +0000 2021,1466452055636201474,RT @JackPosobiec: BREAKING: Epstein Victim Tak...,184,0,NaN
1,Thu Dec 02 16:59:48 +0000 2021,1466452049172697090,Reactions to the Kyle Rittenhouse not guilty v...,0,0,False
2,Thu Dec 02 16:59:46 +0000 2021,1466452041195134977,"RT @JonathanTurley: During the trial, NBC rele...",640,0,False
3,Thu Dec 02 16:59:46 +0000 2021,1466452041136586761,RT @ericgarland: Mark Zuckerberg allows Kyle R...,82,0,False
4,Thu Dec 02 16:59:43 +0000 2021,1466452030000488448,RT @ericgarland: Mark Zuckerberg allows Kyle R...,81,0,False


In [ ]:
print(Twitterdf_Reduced.iloc[0]['text'])

print('\n')
print(Twitterdf_Reduced.iloc[1]['text'])

print('\n')
print(Twitterdf_Reduced.iloc[2]['text'])



RT @JackPosobiec: BREAKING: Epstein Victim Takes Stand in Ghislaine Maxwell Trial

Supreme Court Official Takes Up Roe v Wade Case 

Kyle R…


Reactions to the Kyle Rittenhouse not guilty verdict | Nightline https://t.co/jBOGZHGO2e


RT @JonathanTurley: During the trial, NBC released a statement that some of us found vague and misleading. Now a police video confirms that…


## Getting rid twitter specific content

In [ ]:
import re as re

# Define a function to clean the text
def clean_twittertext(text):
    # Removes #, @, and https:// and http:// from twitter text
    text = re.sub(r'#\w+', '', str(text))  # remove hashtags
    text = re.sub(r'@\w+', '', str(text))  # remove hashtags
    text = re.sub(r'http[s]?://\S+', '', str(text))  # remove hashtags
    return text


In [ ]:
# Removing the unnecessary twitter content
Twitterdf_Reduced['CleanedTwitter'] = Twitterdf_Reduced['text'].apply(clean_twittertext)
Twitterdf_Reduced.head()

,created_at,id,text,retweet_count,favorite_count,possibly_sensitive,CleanedTwitter
0,Thu Dec 02 16:59:50 +0000 2021,1466452055636201474,RT @JackPosobiec: BREAKING: Epstein Victim Tak...,184,0,NaN,RT : BREAKING: Epstein Victim Takes Stand in G...
1,Thu Dec 02 16:59:48 +0000 2021,1466452049172697090,Reactions to the Kyle Rittenhouse not guilty v...,0,0,False,Reactions to the Kyle Rittenhouse not guilty v...
2,Thu Dec 02 16:59:46 +0000 2021,1466452041195134977,"RT @JonathanTurley: During the trial, NBC rele...",640,0,False,"RT : During the trial, NBC released a statemen..."
3,Thu Dec 02 16:59:46 +0000 2021,1466452041136586761,RT @ericgarland: Mark Zuckerberg allows Kyle R...,82,0,False,RT : Mark Zuckerberg allows Kyle Rittenhouse b...
4,Thu Dec 02 16:59:43 +0000 2021,1466452030000488448,RT @ericgarland: Mark Zuckerberg allows Kyle R...,81,0,False,RT : Mark Zuckerberg allows Kyle Rittenhouse b...


Comparing a cleaned review against it's original.

In [ ]:
print(Twitterdf_Reduced.iloc[0]['text'])
print(Twitterdf_Reduced.iloc[0]['CleanedTwitter'])
print('\n')
print(Twitterdf_Reduced.iloc[1]['text'])
print(Twitterdf_Reduced.iloc[1]['CleanedTwitter'])
print('\n')
print(Twitterdf_Reduced.iloc[2]['text'])
print(Twitterdf_Reduced.iloc[2]['CleanedTwitter'])

RT @JackPosobiec: BREAKING: Epstein Victim Takes Stand in Ghislaine Maxwell Trial

Supreme Court Official Takes Up Roe v Wade Case 

Kyle R…
RT : BREAKING: Epstein Victim Takes Stand in Ghislaine Maxwell Trial

Supreme Court Official Takes Up Roe v Wade Case 

Kyle R…


Reactions to the Kyle Rittenhouse not guilty verdict | Nightline https://t.co/jBOGZHGO2e
Reactions to the Kyle Rittenhouse not guilty verdict | Nightline 


RT @JonathanTurley: During the trial, NBC released a statement that some of us found vague and misleading. Now a police video confirms that…
RT : During the trial, NBC released a statement that some of us found vague and misleading. Now a police video confirms that…


In [ ]:
# Define a function to clean the text
def clean(text):
    # Removes all special characters and numericals leaving the alphabets
    text = re.sub('[^A-Za-z]+', ' ', str(text)) 
    return text

In [ ]:
# Removing the non-alphabetic characters
Twitterdf_Reduced['CleanedReviews'] = Twitterdf_Reduced['CleanedTwitter'].apply(clean)
Twitterdf_Reduced.head()

,created_at,id,text,retweet_count,favorite_count,possibly_sensitive,CleanedTwitter,CleanedReviews
0,Thu Dec 02 16:59:50 +0000 2021,1466452055636201474,RT @JackPosobiec: BREAKING: Epstein Victim Tak...,184,0,NaN,RT : BREAKING: Epstein Victim Takes Stand in G...,RT BREAKING Epstein Victim Takes Stand in Ghis...
1,Thu Dec 02 16:59:48 +0000 2021,1466452049172697090,Reactions to the Kyle Rittenhouse not guilty v...,0,0,False,Reactions to the Kyle Rittenhouse not guilty v...,Reactions to the Kyle Rittenhouse not guilty v...
2,Thu Dec 02 16:59:46 +0000 2021,1466452041195134977,"RT @JonathanTurley: During the trial, NBC rele...",640,0,False,"RT : During the trial, NBC released a statemen...",RT During the trial NBC released a statement t...
3,Thu Dec 02 16:59:46 +0000 2021,1466452041136586761,RT @ericgarland: Mark Zuckerberg allows Kyle R...,82,0,False,RT : Mark Zuckerberg allows Kyle Rittenhouse b...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...
4,Thu Dec 02 16:59:43 +0000 2021,1466452030000488448,RT @ericgarland: Mark Zuckerberg allows Kyle R...,81,0,False,RT : Mark Zuckerberg allows Kyle Rittenhouse b...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...


In [ ]:
print(Twitterdf_Reduced.iloc[0]['text'])
print(Twitterdf_Reduced.iloc[0]['CleanedReviews'])
print('\n')
print(Twitterdf_Reduced.iloc[1]['text'])
print(Twitterdf_Reduced.iloc[1]['CleanedReviews'])
print('\n')
print(Twitterdf_Reduced.iloc[2]['text'])
print(Twitterdf_Reduced.iloc[2]['CleanedReviews'])

RT @JackPosobiec: BREAKING: Epstein Victim Takes Stand in Ghislaine Maxwell Trial

Supreme Court Official Takes Up Roe v Wade Case 

Kyle R…
RT BREAKING Epstein Victim Takes Stand in Ghislaine Maxwell Trial Supreme Court Official Takes Up Roe v Wade Case Kyle R 


Reactions to the Kyle Rittenhouse not guilty verdict | Nightline https://t.co/jBOGZHGO2e
Reactions to the Kyle Rittenhouse not guilty verdict Nightline 


RT @JonathanTurley: During the trial, NBC released a statement that some of us found vague and misleading. Now a police video confirms that…
RT During the trial NBC released a statement that some of us found vague and misleading Now a police video confirms that 


## Tokenization and Removal of Stopwords

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

def token_stop(text):
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    filtered_sentence = (" ").join(tokens_without_sw)
    return filtered_sentence


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The following code uses the custom token_stop_pos() function defined above.  This custom function tokenizes the text, removes any stop words, and identifies the parts of speech.  The pos dictionary is specified to keep only adjectives, verbs, nouns, and adverbs.

In [ ]:
Twitterdf_Reduced['CleanedReviews_NoStopWords'] = Twitterdf_Reduced['CleanedReviews'].apply(token_stop)
Twitterdf_Reduced.head()

,created_at,id,text,retweet_count,favorite_count,possibly_sensitive,CleanedTwitter,CleanedReviews,CleanedReviews_NoStopWords
0,Thu Dec 02 16:59:50 +0000 2021,1466452055636201474,RT @JackPosobiec: BREAKING: Epstein Victim Tak...,184,0,NaN,RT : BREAKING: Epstein Victim Takes Stand in G...,RT BREAKING Epstein Victim Takes Stand in Ghis...,RT BREAKING Epstein Victim Takes Stand Ghislai...
1,Thu Dec 02 16:59:48 +0000 2021,1466452049172697090,Reactions to the Kyle Rittenhouse not guilty v...,0,0,False,Reactions to the Kyle Rittenhouse not guilty v...,Reactions to the Kyle Rittenhouse not guilty v...,Reactions Kyle Rittenhouse guilty verdict Nigh...
2,Thu Dec 02 16:59:46 +0000 2021,1466452041195134977,"RT @JonathanTurley: During the trial, NBC rele...",640,0,False,"RT : During the trial, NBC released a statemen...",RT During the trial NBC released a statement t...,RT During trial NBC released statement us foun...
3,Thu Dec 02 16:59:46 +0000 2021,1466452041136586761,RT @ericgarland: Mark Zuckerberg allows Kyle R...,82,0,False,RT : Mark Zuckerberg allows Kyle Rittenhouse b...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...
4,Thu Dec 02 16:59:43 +0000 2021,1466452030000488448,RT @ericgarland: Mark Zuckerberg allows Kyle R...,81,0,False,RT : Mark Zuckerberg allows Kyle Rittenhouse b...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...


## Obtain the root of each word

<strong>Lemmatization</strong> is the process of converting a word to its base form. An alternative process is <strong>stemming</strong> a word which simply removes the last few characters, e.g. the s from studys would be removed by stemming or the ied from studied would be removed by stemming.  Lemmatization is a more sophicated and considers the context and converts the word to its meaningful base form. Lemmatization tends to lead to more meaningful conversation than stemming. 

Once again the natural language toolkit will be used for the lemmitization of our reviews.  The custom lemmatize() function takes the parts of speech list, applies the lemmatization to this list, and then creates a string that contains the base roots of each word.

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    word_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output
   

In [ ]:
Twitterdf_Reduced['ReviewsFinal'] = Twitterdf_Reduced['CleanedReviews_NoStopWords'].apply(lemmatize)
Twitterdf_Reduced.head()

,created_at,id,text,retweet_count,favorite_count,possibly_sensitive,CleanedTwitter,CleanedReviews,CleanedReviews_NoStopWords,ReviewsFinal
0,Thu Dec 02 16:59:50 +0000 2021,1466452055636201474,RT @JackPosobiec: BREAKING: Epstein Victim Tak...,184,0,NaN,RT : BREAKING: Epstein Victim Takes Stand in G...,RT BREAKING Epstein Victim Takes Stand in Ghis...,RT BREAKING Epstein Victim Takes Stand Ghislai...,RT BREAKING Epstein Victim Takes Stand Ghislai...
1,Thu Dec 02 16:59:48 +0000 2021,1466452049172697090,Reactions to the Kyle Rittenhouse not guilty v...,0,0,False,Reactions to the Kyle Rittenhouse not guilty v...,Reactions to the Kyle Rittenhouse not guilty v...,Reactions Kyle Rittenhouse guilty verdict Nigh...,Reactions Kyle Rittenhouse guilty verdict Nigh...
2,Thu Dec 02 16:59:46 +0000 2021,1466452041195134977,"RT @JonathanTurley: During the trial, NBC rele...",640,0,False,"RT : During the trial, NBC released a statemen...",RT During the trial NBC released a statement t...,RT During trial NBC released statement us foun...,RT During trial NBC released statement u found...
3,Thu Dec 02 16:59:46 +0000 2021,1466452041136586761,RT @ericgarland: Mark Zuckerberg allows Kyle R...,82,0,False,RT : Mark Zuckerberg allows Kyle Rittenhouse b...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...
4,Thu Dec 02 16:59:43 +0000 2021,1466452030000488448,RT @ericgarland: Mark Zuckerberg allows Kyle R...,81,0,False,RT : Mark Zuckerberg allows Kyle Rittenhouse b...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...


In [ ]:
print(Twitterdf_Reduced.iloc[0]['text'])
print(Twitterdf_Reduced.iloc[0]['ReviewsFinal'])
print('\n')
print(Twitterdf_Reduced.iloc[1]['text'])
print(Twitterdf_Reduced.iloc[1]['ReviewsFinal'])
print('\n')
print(Twitterdf_Reduced.iloc[2]['text'])
print(Twitterdf_Reduced.iloc[2]['ReviewsFinal'])

RT @JackPosobiec: BREAKING: Epstein Victim Takes Stand in Ghislaine Maxwell Trial

Supreme Court Official Takes Up Roe v Wade Case 

Kyle R…
RT BREAKING Epstein Victim Takes Stand Ghislaine Maxwell Trial Supreme Court Official Takes Up Roe Wade Case Kyle R


Reactions to the Kyle Rittenhouse not guilty verdict | Nightline https://t.co/jBOGZHGO2e
Reactions Kyle Rittenhouse guilty verdict Nightline


RT @JonathanTurley: During the trial, NBC released a statement that some of us found vague and misleading. Now a police video confirms that…
RT During trial NBC released statement u found vague misleading Now police video confirms


## Emotion Analysis

Source:  https://towardsdatascience.com/text2emotion-python-package-to-detect-emotions-from-textual-data-b2e7b7ce1153



In [ ]:
#Install package using pip
!pip install text2emotion

#Import the modules
import text2emotion as te

# function to calculate polarity
def getEmotion(review):
    return te.get_emotion(review)

 

In [ ]:
#Get the emotion scores
Twitterdf_Reduced['Emotion'] = Twitterdf_Reduced['ReviewsFinal'].apply(getEmotion)
Twitterdf_Reduced.head()

,created_at,id,text,retweet_count,favorite_count,possibly_sensitive,CleanedTwitter,CleanedReviews,CleanedReviews_NoStopWords,ReviewsFinal,Emotion
0,Thu Dec 02 16:59:50 +0000 2021,1466452055636201474,RT @JackPosobiec: BREAKING: Epstein Victim Tak...,184,0,NaN,RT : BREAKING: Epstein Victim Takes Stand in G...,RT BREAKING Epstein Victim Takes Stand in Ghis...,RT BREAKING Epstein Victim Takes Stand Ghislai...,RT BREAKING Epstein Victim Takes Stand Ghislai...,"{'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.6, ..."
1,Thu Dec 02 16:59:48 +0000 2021,1466452049172697090,Reactions to the Kyle Rittenhouse not guilty v...,0,0,False,Reactions to the Kyle Rittenhouse not guilty v...,Reactions to the Kyle Rittenhouse not guilty v...,Reactions Kyle Rittenhouse guilty verdict Nigh...,Reactions Kyle Rittenhouse guilty verdict Nigh...,"{'Happy': 1.0, 'Angry': 0.0, 'Surprise': 0.0, ..."
2,Thu Dec 02 16:59:46 +0000 2021,1466452041195134977,"RT @JonathanTurley: During the trial, NBC rele...",640,0,False,"RT : During the trial, NBC released a statemen...",RT During the trial NBC released a statement t...,RT During trial NBC released statement us foun...,RT During trial NBC released statement u found...,"{'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.0, ..."
3,Thu Dec 02 16:59:46 +0000 2021,1466452041136586761,RT @ericgarland: Mark Zuckerberg allows Kyle R...,82,0,False,RT : Mark Zuckerberg allows Kyle Rittenhouse b...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...,"{'Happy': 0, 'Angry': 0, 'Surprise': 0, 'Sad':..."
4,Thu Dec 02 16:59:43 +0000 2021,1466452030000488448,RT @ericgarland: Mark Zuckerberg allows Kyle R...,81,0,False,RT : Mark Zuckerberg allows Kyle Rittenhouse b...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...,"{'Happy': 0, 'Angry': 0, 'Surprise': 0, 'Sad':..."


In [ ]:
#Split emotion list up into seperate columns
Twitterdf_Reduced[['Happy','Angry', 'Surprise', 'Sad', 'Fear']] = pd.DataFrame(Twitterdf_Reduced.Emotion.values.tolist(), index= Twitterdf_Reduced.index)

Twitterdf_Reduced.head()

,created_at,id,text,retweet_count,favorite_count,possibly_sensitive,CleanedTwitter,CleanedReviews,CleanedReviews_NoStopWords,ReviewsFinal,Emotion,Happy,Angry,Surprise,Sad,Fear
0,Thu Dec 02 16:59:50 +0000 2021,1466452055636201474,RT @JackPosobiec: BREAKING: Epstein Victim Tak...,184,0,NaN,RT : BREAKING: Epstein Victim Takes Stand in G...,RT BREAKING Epstein Victim Takes Stand in Ghis...,RT BREAKING Epstein Victim Takes Stand Ghislai...,RT BREAKING Epstein Victim Takes Stand Ghislai...,"{'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.6, ...",0.0,0.0,0.6,0.0,0.4
1,Thu Dec 02 16:59:48 +0000 2021,1466452049172697090,Reactions to the Kyle Rittenhouse not guilty v...,0,0,False,Reactions to the Kyle Rittenhouse not guilty v...,Reactions to the Kyle Rittenhouse not guilty v...,Reactions Kyle Rittenhouse guilty verdict Nigh...,Reactions Kyle Rittenhouse guilty verdict Nigh...,"{'Happy': 1.0, 'Angry': 0.0, 'Surprise': 0.0, ...",1.0,0.0,0.0,0.0,0.0
2,Thu Dec 02 16:59:46 +0000 2021,1466452041195134977,"RT @JonathanTurley: During the trial, NBC rele...",640,0,False,"RT : During the trial, NBC released a statemen...",RT During the trial NBC released a statement t...,RT During trial NBC released statement us foun...,RT During trial NBC released statement u found...,"{'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.0, ...",0.0,0.0,0.0,0.0,1.0
3,Thu Dec 02 16:59:46 +0000 2021,1466452041136586761,RT @ericgarland: Mark Zuckerberg allows Kyle R...,82,0,False,RT : Mark Zuckerberg allows Kyle Rittenhouse b...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...,"{'Happy': 0, 'Angry': 0, 'Surprise': 0, 'Sad':...",0.0,0.0,0.0,0.0,0.0
4,Thu Dec 02 16:59:43 +0000 2021,1466452030000488448,RT @ericgarland: Mark Zuckerberg allows Kyle R...,81,0,False,RT : Mark Zuckerberg allows Kyle Rittenhouse b...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...,RT Mark Zuckerberg allows Kyle Rittenhouse bac...,"{'Happy': 0, 'Angry': 0, 'Surprise': 0, 'Sad':...",0.0,0.0,0.0,0.0,0.0


In [ ]:
#Piping in dfply and using filter_by() to grab requested rows.
(Twitterdf_Reduced
    >>summarize(
                  Avg_Happy=X.Happy.mean(),
                  Avg_Angry=X.Angry.mean(),
                  Avg_Surprise=X.Surprise.mean(),
                  Avg_Sad=X.Sad.mean(),
                  Avg_Fear=X.Fear.mean()
                )
)

,Avg_Happy,Avg_Angry,Avg_Surprise,Avg_Sad,Avg_Fear
0,0.0558,0.1025,0.1678,0.2532,0.2404
